<a href="https://colab.research.google.com/github/Liza-IITP/Dynamic_Pricing_Parking_Lots/blob/main/SmartParking_DynamicPricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Real-time dynamic pricing for 14 urban parking spots.**


*Model 1 (Linear), Model 2 (Demand-Based), Model 3 (Competitive).*

In [1]:

!pip install pathway bokeh panel --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:

import pandas as pd
import numpy as np
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
pn.extension()

In [10]:
# 🚀 Model 1: Linear Price Update Based on Occupancy
# Price(t+1) = Price(t) + alpha * OccupancyRate
# Alpha is estimated using simple linear regression

# ------------------ LOAD DATA ------------------
df = pd.read_csv('data.csv')

# ------------------ FEATURE ENGINEERING ------------------
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

# Handle missing VehicleType categories
vehicle_types = ["bike", "car", "cycle", "truck"]
vehicle_dummies = pd.get_dummies(df["VehicleType"], prefix="VehicleType").astype(int)
for vt in vehicle_types:
    col = f"VehicleType_{vt}"
    if col not in vehicle_dummies:
        vehicle_dummies[col] = 0
df = pd.concat([df, vehicle_dummies], axis=1)

traffic_map = {'low': 0, 'average': 1, 'high': 2}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

df["DayOfWeek"] = df["Timestamp"].dt.dayofweek

df.drop(columns=["Unnamed: 0", "Latitude", "Longitude", "VehicleType",
                 "LastUpdatedDate", "LastUpdatedTime", "TrafficConditionNearby"],
        inplace=True)

df.drop_duplicates(inplace=True)
df = df.sort_values(['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

# Save to CSV for Pathway replay
df.to_csv("parking_stream_all_spots.csv", index=False)


# ------------------ ALPHA ESTIMATION ------------------
df["SimulatedPrice"] = np.nan
spot_alpha_list = []

for spot_id in df["SystemCodeNumber"].unique():
    sub_df = df[df["SystemCodeNumber"] == spot_id].copy()
    base_price = 10
    alpha_dummy = 1.0
    price_series = [base_price]
    for occ in sub_df["OccupancyRate"]:
        price_series.append(price_series[-1] + alpha_dummy * occ)
    sub_df["SimulatedPrice"] = price_series[1:]
    sub_df["DeltaPrice"] = sub_df["SimulatedPrice"].diff()
    sub_df["OccupancyRate_lag"] = sub_df["OccupancyRate"].shift(1)
    sub_df = sub_df.dropna()

    x = sub_df["OccupancyRate_lag"].values
    y = sub_df["DeltaPrice"].values
    alpha = np.sum((x - x.mean()) * (y - y.mean())) / np.sum((x - x.mean())**2)
    spot_alpha_list.append(alpha)

alpha_avg = np.mean(spot_alpha_list)
print(f"📈 Average Estimated Alpha for All Spots: {alpha_avg:.4f}")

# ------------------ PATHWAY STREAM SETUP ------------------
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    OccupancyRate: float
    Capacity: int
    QueueLength: int
    IsSpecialDay: int
    TrafficLevel: int
    DayOfWeek: int
    VehicleType_bike: int
    VehicleType_car: int
    VehicleType_cycle: int
    VehicleType_truck: int

data = pw.demo.replay_csv("parking_stream_all_spots.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t=data.Timestamp.dt.strptime(fmt),
    day=data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# ------------------ MODEL-1: DAILY PRICE BY SPOT ------------------
model1_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=(pw.this.SystemCodeNumber, pw.this.day),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        SystemCodeNumber=pw.reducers.any(pw.this.SystemCodeNumber),
        total_occ=pw.reducers.sum(pw.this.OccupancyRate),
    )
    .with_columns(
        Price_Model1 = 10 + alpha_avg * pw.this.total_occ
    )
)



# ------------------ VISUALIZATION ------------------
def plot_model1(source):
    fig = bokeh.plotting.figure(
        height=500,
        width=900,
        title="Model-1: Dynamic Price for All 14 Parking Spots (Daily)",
        x_axis_type="datetime"
    )
    for code in source.data["SystemCodeNumber"]:
        fig.scatter("t", "Price_Model1", source=source, size=5, color="navy", legend_label="Dynamic Price")
    fig.xaxis.axis_label = "Date"
    fig.yaxis.axis_label = "Price"
    return fig

viz_model1 = model1_window.plot(plot_model1, sorting_col="t")
pn.Column(viz_model1).servable()

📈 Average Estimated Alpha for All Spots: 0.8967


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [5]:
#  Model 2: Demand-Based Dynamic Pricing
# Demand = α*(Occ/Cap) + β*Queue - γ*Traffic + δ*IsSpecialDay + ε*VehicleTypeWeight + τ*Trend
# Price = BasePrice * (1 + λ * NormalizedDemand)


df = pd.read_csv("data.csv")
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]
vehicle_dummies = pd.get_dummies(df["VehicleType"], prefix="VehicleType").astype(int)
vehicle_types = ["bike", "car", "cycle", "truck"]
for vt in vehicle_types:
    if f"VehicleType_{vt}" not in vehicle_dummies:
        vehicle_dummies[f"VehicleType_{vt}"] = 0
df = pd.concat([df, vehicle_dummies], axis=1)

vehicle_weights = {
    "VehicleType_bike": 0.2,
    "VehicleType_car": 1.0,
    "VehicleType_cycle": 0.1,
    "VehicleType_truck": 1.5
}
df["VehicleTypeWeight"] = sum(df[col] * wt for col, wt in vehicle_weights.items())

# Map traffic
traffic_map = {'low': 0, 'average': 1, 'high': 2}
df["TrafficLevel"] = df["TrafficConditionNearby"].map(traffic_map)

# Time features
df["DayOfWeek"] = df["Timestamp"].dt.dayofweek
df["TimeSlot"] = df["Timestamp"].dt.hour + df["Timestamp"].dt.minute / 60

def slot_weight(t):
    if 8 <= t < 10: return 0.8
    elif 10 <= t < 12: return 1.2
    elif 12 <= t < 14: return 1.5
    elif 14 <= t < 16: return 1.2
    else: return 1.0

df["SlotWeight"] = df["TimeSlot"].apply(slot_weight)

# Zone weight (mock rule: lat > 25.6 → downtown)
df["ZoneWeight"] = df["Latitude"].apply(lambda lat: 1.5 if lat > 25.6 else 1.0)

# Moving average and trend
df["OccRate_MA3"] = df.groupby("SystemCodeNumber")["OccupancyRate"].transform(lambda x: x.rolling(3, min_periods=1).mean())
df["OccRate_Trend"] = df.groupby("SystemCodeNumber")["OccupancyRate"].diff().fillna(0)

α, β, γ, δ, ε, τ = 1.0, 0.5, 0.3, 1.5, 1.0, 0.2
λ = 0.5
BasePrice = 10

df["RawDemand"] = (
    α * df["OccRate_MA3"] +
    β * df["QueueLength"] -
    γ * df["TrafficLevel"] +
    δ * df["IsSpecialDay"] +
    ε * df["VehicleTypeWeight"] +
    τ * df["OccRate_Trend"]
)
df["RawDemand"] *= df["SlotWeight"]
df["RawDemand"] *= df["ZoneWeight"]

# Normalize
min_d, max_d = df["RawDemand"].min(), df["RawDemand"].max()
df["NormalizedDemand"] = (df["RawDemand"] - min_d) / (max_d - min_d)
df["NormalizedDemand"] = df["NormalizedDemand"].clip(0, 1)
df["Model2Price"] = BasePrice * (1 + λ * df["NormalizedDemand"])
df["Model2Price_Smoothed"] = df.groupby("SystemCodeNumber")["Model2Price"].transform(lambda x: x.ewm(alpha=0.3).mean())

df.to_csv("parking_stream_model2.csv", index=False)
class Model2Schema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    OccupancyRate: float
    Capacity: int
    QueueLength: int
    IsSpecialDay: int
    TrafficLevel: int
    DayOfWeek: int
    VehicleType_bike: int
    VehicleType_car: int
    VehicleType_cycle: int
    VehicleType_truck: int
    VehicleTypeWeight: float
    OccRate_MA3: float
    OccRate_Trend: float
    SlotWeight: float
    ZoneWeight: float
    RawDemand: float
    NormalizedDemand: float
    Model2Price: float
    Model2Price_Smoothed: float

data = pw.demo.replay_csv("parking_stream_model2.csv", schema=Model2Schema, input_rate=1000)
fmt = "%Y-%m-%d %H:%M:%S"
data_time = data.with_columns(
    t=data.Timestamp.dt.strptime(fmt),
    day=data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"))

window = (
    data_time.windowby(
        pw.this.t,
        instance=(pw.this.SystemCodeNumber, pw.this.day),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        SystemCodeNumber=pw.reducers.any(pw.this.SystemCodeNumber),
        sum_price=pw.reducers.sum(pw.this.Model2Price_Smoothed),
        count_price=pw.reducers.count(pw.this.Model2Price_Smoothed),
    )
    .with_columns(
        AvgPrice_Model2 = pw.this.sum_price / pw.this.count_price
    )
)



def plot(source):
    fig = bokeh.plotting.figure(
        height=450, width=900,
        title="Model 2 (Enhanced): Smoothed Demand-Based Price per Spot",
        x_axis_type="datetime"
    )
    fig.line("t", "AvgPrice_Model2", source=source, line_width=2, color="green")
    fig.scatter("t", "AvgPrice_Model2", source=source, size=6, color="red")
    fig.xaxis.axis_label = "Date"
    fig.yaxis.axis_label = "Avg Price"
    return fig

viz = window.plot(plot, sorting_col="t")
pn.panel(viz).servable()

Column
    [0] Row
        [0] Markdown(str)
        [1] TooltipIcon(value='Table depends o...)
    [1] Bokeh(figure)

In [7]:
#  Model 3: Competitive Pricing Based on Nearby Spots
# Uses Haversine distance and compares prices with nearby spots
# Competitive adjustment applied to Model 2 prices
df = pd.read_csv("data.csv")
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

vehicle_dummies = pd.get_dummies(df["VehicleType"], prefix="VehicleType").astype(int)
for vt in ["bike", "car", "cycle", "truck"]:
    if f"VehicleType_{vt}" not in vehicle_dummies:
        vehicle_dummies[f"VehicleType_{vt}"] = 0
df = pd.concat([df, vehicle_dummies], axis=1)

weights = {"VehicleType_bike": 0.2, "VehicleType_car": 1.0, "VehicleType_cycle": 0.1, "VehicleType_truck": 1.5}
df["VehicleTypeWeight"] = sum(df[col] * wt for col, wt in weights.items())

df["TrafficLevel"] = df["TrafficConditionNearby"].map({'low': 0, 'average': 1, 'high': 2})
df["DayOfWeek"] = df["Timestamp"].dt.dayofweek
df["TimeSlot"] = df["Timestamp"].dt.hour + df["Timestamp"].dt.minute / 60

def slot_weight(t): return 1.5 if 12 <= t < 14 else (1.2 if 10 <= t < 12 or 14 <= t < 16 else (0.8 if 8 <= t < 10 else 1.0))
df["SlotWeight"] = df["TimeSlot"].apply(slot_weight)
df["ZoneWeight"] = df["Latitude"].apply(lambda lat: 1.5 if lat > 25.6 else 1.0)

df["OccRate_MA3"] = df.groupby("SystemCodeNumber")["OccupancyRate"].transform(lambda x: x.rolling(3, min_periods=1).mean())
df["OccRate_Trend"] = df.groupby("SystemCodeNumber")["OccupancyRate"].diff().fillna(0)

α, β, γ, δ, ε, τ, λ = 1.0, 0.5, 0.3, 1.5, 1.0, 0.2, 0.5
BasePrice = 10

df["RawDemand"] = (
    α * df["OccRate_MA3"] + β * df["QueueLength"] - γ * df["TrafficLevel"] +
    δ * df["IsSpecialDay"] + ε * df["VehicleTypeWeight"] + τ * df["OccRate_Trend"]
) * df["SlotWeight"] * df["ZoneWeight"]

df["NormalizedDemand"] = ((df["RawDemand"] - df["RawDemand"].min()) /
                          (df["RawDemand"].max() - df["RawDemand"].min())).clip(0, 1)

df["Model2Price"] = BasePrice * (1 + λ * df["NormalizedDemand"])
df["Model2Price_Smoothed"] = df.groupby("SystemCodeNumber")["Model2Price"].transform(lambda x: x.ewm(alpha=0.3).mean())


def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    return R * 2 * np.arcsin(np.sqrt(np.sin((lat2-lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2-lon1)/2)**2))

def adjust_price(row, df, radius=0.5):
    nearby = df[(df["Timestamp"] == row["Timestamp"]) & (df["SystemCodeNumber"] != row["SystemCodeNumber"])].copy()
    nearby["Distance"] = haversine(row["Latitude"], row["Longitude"], nearby["Latitude"], nearby["Longitude"])
    nearby = nearby[nearby["Distance"] <= radius]
    if nearby.empty: return row["Model2Price_Smoothed"]
    comp = nearby["Model2Price_Smoothed"].mean()
    if row["OccupancyRate"] >= 0.9 and comp < row["Model2Price_Smoothed"]: return row["Model2Price_Smoothed"] - 1
    if comp > row["Model2Price_Smoothed"] + 1: return row["Model2Price_Smoothed"] + 1
    return row["Model2Price_Smoothed"]

df["Model3Price_Competitive"] = df.apply(lambda row: adjust_price(row, df), axis=1)
df["SuggestReroute"] = df.apply(lambda r: r["OccupancyRate"] >= 0.95 and r["Model3Price_Competitive"] > 12, axis=1)

df.to_csv("parking_stream_model3.csv", index=False)

class Model3Schema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    OccupancyRate: float
    QueueLength: int
    IsSpecialDay: int
    TrafficLevel: int
    DayOfWeek: int
    VehicleType_bike: int
    VehicleType_car: int
    VehicleType_cycle: int
    VehicleType_truck: int
    VehicleTypeWeight: float
    OccRate_MA3: float
    OccRate_Trend: float
    SlotWeight: float
    ZoneWeight: float
    RawDemand: float
    NormalizedDemand: float
    Model2Price_Smoothed: float
    Model3Price_Competitive: float
    SuggestReroute: bool

data = pw.demo.replay_csv("parking_stream_model3.csv", schema=Model3Schema, input_rate=1000)
fmt = "%Y-%m-%d %H:%M:%S"
data_time = data.with_columns(
    t=data.Timestamp.dt.strptime(fmt),
    day=data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

window = (
    data_time.windowby(
        pw.this.t,
        instance=(pw.this.SystemCodeNumber, pw.this.day),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        SystemCodeNumber=pw.reducers.any(pw.this.SystemCodeNumber),
        sum_price=pw.reducers.sum(pw.this.Model3Price_Competitive),
        count_price=pw.reducers.count(pw.this.Model3Price_Competitive),
    )
    .with_columns(
        AvgPrice_Model3 = pw.this.sum_price / pw.this.count_price
    )
)

def plot(source):
    fig = bokeh.plotting.figure(height=450, width=900, title="Model 3: Competitive Pricing", x_axis_type="datetime")
    fig.line("t", "AvgPrice_Model3", source=source, line_width=2, color="orange")
    fig.scatter("t", "AvgPrice_Model3", source=source, size=6, color="red")
    return fig

viz = window.plot(plot, sorting_col="t")
pn.panel(viz).servable()


Column
    [0] Row
        [0] Markdown(str)
        [1] TooltipIcon(value='Table depends o...)
    [1] Bokeh(figure)

In [8]:
# Run Pathway Engine
%%capture --no-display
pw.run()

Output()

## 📊 Visualizations
Each model includes Bokeh plots rendered with Panel, supporting real-time, time-series pricing visualization.

- Model 1: Price vs Time (per spot)
- Model 2: Demand-weighted Price
- Model 3: Competitive-adjusted Price


## 📘 Assumptions
- Linear behavior in demand and occupancy
- Haversine used for local competitor comparison
- Manual coefficients tuned for base simulation (can be learned dynamically later)


## ✅ Conclusion
- Shows a progression from basic to intelligent pricing logic
- Uses modern streaming + visualization tech (Pathway + Bokeh)
- Real-time adaptable for smart city use cases